## Example script illustrating sound classification on audio stream
This notebook is an example how to use DeGirum PySDK to do sound classification AI inference of an audio stream from local microphone.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

**pyaudio package with portaudio is required to run this sample.**

**Access to microphone is required to run this sample.**

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

cloud_token = mytools.get_token() # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url() # get cloud zoo URL from env.ini file

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

# 1. Inference on the DeGirum Cloud Platform
zoo = dg.connect_model_zoo("dgcps://cs.degirum.com" + cloud_zoo_url, cloud_token)

# 2. Inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN
# zoo = dg.connect_model_zoo((mytools.get_ai_server_hostname(), "https://cs.degirum.com" + cloud_zoo_url), cloud_token)

# 3. Inference on DeGirum ORCA accelerator installed on your computer
# zoo = dg.connect_model_zoo("https://cs.degirum.com" + cloud_zoo_url, cloud_token)


#### The rest of the cells below should run without any modifications

In [ ]:
import sys
import numpy as np
from IPython.display import clear_output

In [ ]:
# load YAMNET sound classification model for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v1_yamnet_sound_cls--96x64_quant_n2x_orca_1")

In [ ]:
abort = False # stream abort flag
N = 5 # inference results history depth
history = [] # list of N consecutive inference results

sampling_rate_hz = model.model_info.InputSamplingRate[0]
read_buffer_size = model.model_info.InputWaveformSize[0] // 2 # two read buffers in waveform for half-length overlapping

# Acquire model input stream object
with mytools.open_audio_stream(sampling_rate_hz, read_buffer_size) as stream:
    #
    # AI prediction loop.
    # emit keyboard typing sound to stop
    #
    for res in model.predict_batch(mytools.audio_overlapped_source(stream, lambda: abort)):
        # clear Jupyter output cell
        clear_output(wait = True) 
        
        # add top inference result to history
        history.insert(0, f"{res.results[0]['label']}: {res.results[0]['score']}" )
    
        # keep only N last elements in history
        if len(history) > N:
            history.pop()
    
        # print history
        for m in history:
            print(m)
        
        # check for stop condition
        if res.results[0]['label'] == "Typing":
            abort = True
    